In [1]:
import json
import pandas as pd

In [2]:
test_file_template = '''
import time
import unittest

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager


class FormTests(unittest.TestCase):
\tdef setUp(self):
\t\tself.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
\t\tself.driver.get("{url}")
    
    
\tdef tearDown(self):
\t\tself.driver.close()
    
    
{test_cases}


if __name__ == "__main__":
\tunittest.main()
'''


test_case_template = '''
\tdef test_{name}(self):
\t\t\'\'\'{description}\'\'\'
\t\tdriver = self.driver
{code}
'''


send_key_template = '''
\t\telement = WebDriverWait(driver, 5).until(
\t\t\tEC.presence_of_element_located((By.XPATH, "{xpath}"))
\t\t)
\t\telement.send_keys("{value}")
'''


submit_template = '''
\t\tbutton = WebDriverWait(driver, 5).until(
\t\t\tEC.presence_of_element_located((By.XPATH, "{form_xpath}//*[@type='submit']"))
\t\t)
\t\tdriver.execute_script('arguments[0].click()', button)
'''

assertion_template = '''
\t\ttime.sleep(1)
\t\tbody_text = driver.find_element(By.TAG_NAME, 'body').text
\t\tassert "{feedback}" in body_text
'''

In [3]:
APP_NAME = 'Air Canada'
APP_URL = 'https://www.aircanada.com/ca/en/aco/home.html'
FORM_XPATH = '//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]'


In [4]:
values_df = pd.read_csv('./random.csv')

In [5]:
values_df

,changes,fuzz_type,values
0,Please select a valid destination for this tri...,empty,{'//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...
1,Please select a valid point of origin for this...,empty,{'//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...


In [6]:
test_cases = []

for idx, row in values_df.iterrows():
    values = json.loads(row['values'].replace('\'', '"').lower())
    
    fields = []
    for xpath, value in values.items():
        send_key = send_key_template.format(
            xpath=xpath,
            value=value
        )
        fields.append(send_key)
    
    submit = submit_template.format(
        form_xpath=FORM_XPATH
    )
    fields.append(submit)
    
    # TODO: add assertions on url
    # TODO: create a more unique string to split feedback
    assertions = []
    for feedback in row['changes'].split('-'):
        assertion = assertion_template.format(feedback=feedback)
        assertions.append(assertion)
        
    
    test_case = test_case_template.format(
        name=idx,
        description=idx,
        code='\n'.join([*fields, *assertions])
    )
    
    test_cases.append(test_case)

test_file = test_file_template.format(
    url=APP_URL,
    test_cases='\n'.join(test_cases)
)

In [7]:
with open('generated_test.py', 'w+') as f:
    f.write(test_file.strip())